In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from matplotlib.pyplot import plot

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
with open('../input/micro-breast/test_data.pickle', 'rb') as fptr:
     test_data = pickle.load(fptr)
with open('../input/micro-breast/test_labels.pickle', 'rb') as fptr:
     test_labels = pickle.load(fptr)
with open('../input/micro-breast/test_md.pickle', 'rb') as fptr:
     test_md = pickle.load(fptr)
with open('../input/micro-breast/train_data.pickle', 'rb') as fptr:
     train_data = pickle.load(fptr)
with open('../input/micro-breast/train_labels.pickle', 'rb') as fptr:
     train_labels = pickle.load(fptr)
with open('../input/micro-breast/train_md.pickle', 'rb') as fptr:
     train_md = pickle.load(fptr)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
def normalize_samples(data):
    """Normalizes each sample in data to have unity maximum
    Parameters
    ----------
    data : array_like
        3D array of the features for each sample (assumes 2D features)
    Returns
    -------
    normalized_data : array_like
        Array of the features for each sample, normalized so that the
        max value is unity for each sample
    """

    # Assert that data must be 3D
    assert len(np.shape(data)) == 3, 'Error: data must have 3 dim'

    normalized_data = np.ones_like(data)  # Init array to return

    # For each sample
    for sample_idx in range(np.size(data, axis=0)):

        # Normalize to have max of unity
        normalized_data[sample_idx, :, :] = (data[sample_idx, :, :] /
                                             np.max(data[sample_idx, :, :]))

    return normalized_data

In [ ]:
# Convert each train sample from the frequency-domain to the time-domain
for sample_idx in range(np.size(train_data, axis=0)):
    train_data[sample_idx, :, :] = np.fft.ifft(train_data[sample_idx, :, :],
                                               axis=0)
# Convert each test sample from the frequency-domain to the time-domain
for sample_idx in range(np.size(test_data, axis=0)):
    test_data[sample_idx, :, :] = np.fft.ifft(test_data[sample_idx, :, :],
                                              axis=0)

# Take the abs-value of ecah sample, and apply the time-domain window
train_data = np.abs(train_data[:, 5:40, :])
test_data = np.abs(test_data[:, 5:40, :])

# Normalize each sample to have a maximum of unity
train_data = normalize_samples(train_data)
test_data = normalize_samples(test_data)
train_data = np.reshape(train_data, [np.size(train_data, axis=0), 35 * 72])
test_data = np.reshape(test_data, [np.size(test_data, axis=0), 35 * 72])

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
 
        # Define the logistic regression classifier
logreg = LogisticRegression()

        # Fit the classifier
logreg.fit(train_data, train_labels)

ac = accuracy_score(test_labels,logreg.predict(test_data))
#86%
print('Accuracy is: ',ac)
cm = confusion_matrix(test_labels,logreg.predict(test_data))
sns.heatmap(cm,annot=True,fmt="d")

sensitivity=cm[0,0]/ (cm[0,0]+ cm[1,0])
print("Sensitivity is",sensitivity)
specificity=cm[1,1]/ (cm[0,1]+cm[1,1])
print("Specificity is", specificity)
roc_auc_score(test_labels, logreg.predict(test_data))
logreg.get_params()


In [ ]:
cm


In [ ]:
dt= tree.DecisionTreeClassifier(max_depth=15, max_leaf_nodes=100)

        # Fit the classifier
dt.fit(train_data, train_labels)

ac = accuracy_score(test_labels,dt.predict(test_data))
#82+-4%
print('Accuracy is: ',ac)
cm = confusion_matrix(test_labels,dt.predict(test_data))
sns.heatmap(cm,annot=True,fmt="d")
sensitivity=cm[0,0]/ (cm[0,0]+ cm[1,0])
print("Sensitivity is",sensitivity)
specificity=cm[1,1]/ (cm[0,1]+cm[1,1])
print("Specificity is", specificity)
print(roc_auc_score(test_labels, dt.predict(test_data)))
dt.get_params()

In [ ]:
from sklearn import tree
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dt, 
                   filled=True)

In [ ]:
rf= RandomForestClassifier()

        # Fit the classifier
rf.fit(train_data, train_labels)

ac = accuracy_score(test_labels,rf.predict(test_data))
#90+-4%
print('Accuracy is: ',ac)
cm = confusion_matrix(test_labels,rf.predict(test_data))
sns.heatmap(cm,annot=True,fmt="d")
sensitivity=cm[0,0]/ (cm[0,0]+ cm[1,0])
print("Sensitivity is",sensitivity)
specificity=cm[1,1]/ (cm[0,1]+cm[1,1])
print("Specificity is", specificity)
print(roc_auc_score(test_labels,rf.predict(test_data)))
rf.get_params()

In [ ]:
from sklearn.model_selection import learning_curve


In [ ]:
xgb= XGBClassifier(max_depth=7)
xgb.fit(train_data, train_labels)
ac = accuracy_score(test_labels,xgb.predict(test_data))
print('Accuracy is: ',ac)
cm = confusion_matrix(test_labels,xgb.predict(test_data))
sns.heatmap(cm,annot=True,fmt="d")
sensitivity=cm[0,0]/ (cm[0,0]+ cm[1,0])
print("Sensitivity is",sensitivity)
specificity=cm[1,1]/ (cm[0,1]+cm[1,1])
print("Specificity is", specificity)
print(roc_auc_score(test_labels, xgb.predict(test_data)))
xgb.get_params()